In [1]:
import argparse
import json
import os
import sys
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm import tqdm

# Add module path to sys.path
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)

# Import third-party libraries
from config import SciFactT5Config
from multivers.data_r import ClaimDataLoaderGenerator, DataLoaderGenerator, get_dataloader
from multivers.model_r import MultiVerSModel
from multivers import util

# Other necessary imports
import definitions

# Add necessary paths to sys.path
sys.path.append(os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))

# Import project-specific modules
from T5ParEvo.src.data.data import Claim, ClaimPredictions, GoldDataset, Label
from T5ParEvo.src.linguistic.ner_abbr import Abbreviation, NEREntity
from T5ParEvo.target_system.multivers.multivers_interface import ModelPredictorMultivers, PredictionParams,ModelPredictorMultiversList

# Print module_path and definitions.PROJECT_VARS.ROOT_DIR
print('module_path:', module_path)
print('definitions.PROJECT_VARS.ROOT_DIR:', definitions.PROJECT_VARS.ROOT_DIR)


root dir :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo


/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."


/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
module_path: /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
definitions.PROJECT_VARS.ROOT_DIR: /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo


In [2]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Data

In [3]:
# This dataset to be used only for training
cfg= SciFactT5Config()
ds_train = GoldDataset(cfg.target_dataset.loc_target_dataset_corpus,
                    cfg.target_dataset.loc_target_dataset_train)
claim_train = ds_train.get_claim(39)

In [10]:

params = PredictionParams(
    checkpoint_path= "/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/checkpoints/scifact.ckpt",
    output_file= None,#"prediction/pred_opt_scifact.jsonl",
    batch_size=5,
    device=0,
    num_workers=4,
    no_nei=False,
    force_rationale=False,
    debug=False,
    corpus_file = cfg.target_dataset.loc_target_dataset_corpus
)
corpus_file = cfg.target_dataset.loc_target_dataset_corpus#cfg.target_dataset.loc_target_dataset_test#"/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/corpus.jsonl"
    

gold_claims = []
claims_path = cfg.target_dataset.loc_target_dataset_test#'/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/claims_test_retrived.jsonl'
with open(claims_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        claim = Claim(id = data['id'], claim = data['claim'], cited_docs = data['doc_ids'], evidence = {},release = None)
        gold_claims.append(claim)

#get unique claims
unique_gold_claims = Claim.get_unique_claims(gold_claims)

## Predict Original Claims

In [11]:
# Loading unique claims and preparing prediction model
unique_gold_claims = Claim.get_unique_claims(gold_claims)
dataloader_generator = ClaimDataLoaderGenerator(params, unique_gold_claims[0], corpus_file)
dataloader = dataloader_generator.get_dataloader_by_single_claim()
# prediction_model = ModelPredictorMultivers(params, dataloader, corpus_file)
prediction_model = ModelPredictorMultivers(params, unique_gold_claims[0])
original_claim_predictions_raw = prediction_model.predict(unique_gold_claims[0])

Some weights of the model checkpoint at allenai/longformer-large-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# Predicting for unique claims
all_original_claim_predictions : List[ClaimPredictions]= []
for cur_uniq_claim in tqdm(unique_gold_claims[:10], desc="Predicting for unique claims"):
    original_claim_prediction = prediction_model.predict(cur_uniq_claim)
    all_original_claim_predictions.append(original_claim_prediction)

Predicting for unique claims: 100%|██████████| 297/297 [19:30<00:00,  3.94s/it]


In [13]:
# Checking how many claims have more than one prediction
for cur_prediction in all_original_claim_predictions:
    if len(cur_prediction.predictions.keys()) > 1:
        print(cur_prediction)

Predictions for 84: Adult tissue-resident macrophages are maintained independent of circulating monocytes.
Predictions for 107: Amitriptyline ineffective as a treatment for chronic tension-type headaches.
Predictions for 260: Chronic aerobic exercise alters endothelial function, impairing vasodilating mechanisms mediated by NO.
Predictions for 307: DUSP4 downregulation activates the Ras-ERK pathway
Predictions for 465: Genomic instability in leukemia cells results from an increase in reactive oxygen species from oncogene activation.
Predictions for 512: High cardiopulmonary fitness causes decreased mortality rate.
Predictions for 534: Hypertension can be accurately diagnosed with ambulatory blood pressure monitoring.
Predictions for 599: Incidence rates of cervical cancer have increased over time.
Predictions for 833: NLRP12 is a positive regulator of T-cell receptor activation.
Predictions for 864: Notch signalling can occur between tumor cells and stromal cells.
Predictions for 865: 

In [18]:
print(len(all_original_claim_predictions[11].predictions.keys()))
print(all_original_claim_predictions[11].predictions)

4
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4]), 22406695: PredictedAbstract(abstract_id=22406695, label=<Label.SUPPORTS: 2>, rationale=[1]), 22973574: PredictedAbstract(abstract_id=22973574, label=<Label.SUPPORTS: 2>, rationale=[3])}


In [15]:
# #get unique claims
# unique_gold_claims = Claim.get_unique_claims(gold_claims)
# # Predict for unique claims
# corpus_file="/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/corpus.jsonl"
# dataloader_generator = DataLoaderGenerator(params, unique_gold_claims, corpus_file)
# dataloader = dataloader_generator.get_dataloader_by_claims()
# # prediction_model = ModelPredictorMultivers(params, dataloader,corpus_file)
# prediction_model = ModelPredictorMultivers(params, unique_gold_claims[0])
# original_claim_predictions_raw = prediction_model.predict()

In [16]:
# prediction_formatted
# org_claim_predictions: List[ClaimPredictions] = []
# # format all the predictions
# for cur_prediction in tqdm(original_claim_predictions_raw, desc="Formatting predictions"):
#     cur_claim = Claim.get_claim_by_id(gold_claims, cur_prediction['id'])
#     claim_predictions = ClaimPredictions.from_formatted_prediction(cur_prediction, gold_claim = cur_claim)
#     org_claim_predictions.append(claim_predictions)

## Paraphrase and attack original claims

In [19]:
import torch
from transformers import T5ForConditionalGeneration, PreTrainedTokenizer, PreTrainedModel, T5Tokenizer, T5ForConditionalGeneration
# Constants and configurations
paraphrase_model_checkpoint_path = '/home/qudratealahyratu/research/nlp/fact_checking/my_work/SciMedAttack/results/t5_paws_masked_claim_abstract_paws_3_epoch_2/models/model_3_epochs/'
paraphrase_model_tokenizer = 'Vamsi/T5_Paraphrase_Paws'
PARAPHRASE_CONFIG_PARAMS = {
    'max_length': 512,
    'do_sample': True,
    'top_k': 50,
    'top_p': 0.99,
    'repetition_penalty': 3.5,
    'early_stopping': True,
    'num_return_sequences': 10
}

# Utility function to load T5 model
def load_t5_model(checkpoint_path):
    model_t5 = T5ForConditionalGeneration.from_pretrained(checkpoint_path)
    model_t5 = model_t5.to(DEVICE)
    return model_t5


**Technical terms**

In [20]:
import pickle

with open('../data/meta/merged_abbreviations.pkl', 'rb') as f:
    merged_abbreviations = pickle.load(f)

with open('../data/meta/merged_entities.pkl', 'rb') as f:
    merged_entities = pickle.load(f)


**Paraphrase,Entailment,classification model**

In [25]:
# Preparing paraphraser
from T5ParEvo.src.paraphrase.paraphrase_claim import ParaphrasedAttack
from T5ParEvo.src.paraphrase.paraphraser import T5Paraphraser, ModelConfig

# Load T5 model and tokenizer
model_t5 = load_t5_model(paraphrase_model_checkpoint_path)
tokenizer_t5 = T5Tokenizer.from_pretrained(paraphrase_model_tokenizer)

# Initialize paraphrase model and paraphrase attack
paraphrase_config = ModelConfig(**PARAPHRASE_CONFIG_PARAMS)
paraphrase_model = T5Paraphraser(model_t5, tokenizer_t5, paraphrase_config)

# Initialize entailment model
from T5ParEvo.src.paraphrase.paraphrase_claim  import TorchEntailmentPredictionModel


entailment_model = TorchEntailmentPredictionModel(model_path=  'pytorch/fairseq', model_name = 'roberta.large.mnli', device= DEVICE)


# Initialize paraphrase attack
paraphrase_attack = ParaphrasedAttack(paraphrase_model, prediction_model,entailment_model ,list_ners = merged_entities)

Using cache found in /home/qudratealahyratu/.cache/torch/hub/pytorch_fairseq_main
2023-07-14 22:18:33 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz from cache at /home/qudratealahyratu/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/hydra/experimental/initialize.py:48: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  caller_stack_depth=caller_stack_depth + 1,
/home/qudratealahyratu/anaco

In [26]:
result = paraphrase_attack.attack(iteration = 0, original_claim= all_original_claim_predictions[11].gold, 
        original_prediction =all_original_claim_predictions[11].predictions, 
        predict_if_pass_filter=True)

TypeError: expected string or bytes-like object

In [ ]:
for cur_res in result:
    print(cur_res.paraphrased_prediction.predictions)

In [16]:
type(merged_entities[8][0])

T5ParEvo.src.linguistic.ner_abbr.NEREntity

In [ ]:
ParaphrasedClaim

In [17]:
merged_entities

{7: [NEREntity(claim_id=7, ner_text='people', ner_label='ORGANISM', ner_model='ner_bionlp13cg_md', start_char=10, end_char=16)],
 8: [NEREntity(claim_id=8, ner_text='patients', ner_label='ORGANISM', ner_model='ner_bionlp13cg_md', start_char=7, end_char=15),
  NEREntity(claim_id=8, ner_text='melanoma', ner_label='DISEASE', ner_model='ner_bc5cdr_md', start_char=21, end_char=29),
  NEREntity(claim_id=8, ner_text='PD-1', ner_label='GENE_OR_GENE_PRODUCT', ner_model='ner_bionlp13cg_md', start_char=59, end_char=63),
  NEREntity(claim_id=8, ner_text='melanoma', ner_label='DISEASE', ner_model='ner_bc5cdr_md', start_char=112, end_char=120)],
 16: [NEREntity(claim_id=16, ner_text='patients', ner_label='ORGANISM', ner_model='ner_bionlp13cg_md', start_char=7, end_char=15),
  NEREntity(claim_id=16, ner_text='myofibroblasts', ner_label='CELL', ner_model='ner_bionlp13cg_md', start_char=63, end_char=77)],
 23: [NEREntity(claim_id=23, ner_text='patients', ner_label='ORGANISM', ner_model='ner_bionlp13cg_

In [ ]:
# assuming the models and a Claim instance are already initialized
paraphrase_model = Paraphraser()
prediction_model = ModelPredictor()
entailment_checker = EntailmentChecker(PARAPHRASE_PROJECT_SETTINGS['entailment_model']['model_path'], 
                                       PARAPHRASE_PROJECT_SETTINGS['entailment_model']['model_name'], 
                                       device)

original_claim = Claim(...) # initialize with appropriate values
original_prediction = prediction_model.predict(original_claim)

attack = ParaphrasedAttack(paraphrase_model, prediction_model, entailment_checker)
paraphrased_claims = attack.attack(1, original_claim, original_prediction)


Some weights of the model checkpoint at allenai/longformer-large-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Metric` was deprecated since v1.3.0 in favor of `torchmetrics.metric.Metric`. It will be removed in v1.5.0.
  stream(template_mgs % m

Predicting for unique claims: 100%|██████████| 297/297 [18:19<00:00,  3.70s/it]


Formatting predictions: 100%|██████████| 297/297 [00:00<00:00, 355713.39it/s]

Predictions for 84: Adult tissue-resident macrophages are maintained independent of circulating monocytes.
Predictions for 107: Amitriptyline ineffective as a treatment for chronic tension-type headaches.
Predictions for 260: Chronic aerobic exercise alters endothelial function, impairing vasodilating mechanisms mediated by NO.
Predictions for 307: DUSP4 downregulation activates the Ras-ERK pathway
Predictions for 465: Genomic instability in leukemia cells results from an increase in reactive oxygen species from oncogene activation.
Predictions for 512: High cardiopulmonary fitness causes decreased mortality rate.
Predictions for 534: Hypertension can be accurately diagnosed with ambulatory blood pressure monitoring.
Predictions for 599: Incidence rates of cervical cancer have increased over time.
Predictions for 833: NLRP12 is a positive regulator of T-cell receptor activation.
Predictions for 864: Notch signalling can occur between tumor cells and stromal cells.
Predictions for 865: 

In [33]:
all_original_claim_predictions[11].predictions # has more than one prediction

{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]),
 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4]),
 22406695: PredictedAbstract(abstract_id=22406695, label=<Label.SUPPORTS: 2>, rationale=[1]),
 22973574: PredictedAbstract(abstract_id=22973574, label=<Label.SUPPORTS: 2>, rationale=[3])}

In [7]:
#NONE
# Formatting all the predictions
org_claim_predictions = []
cur_prediction =original_claim_predictions_raw
cur_claim = Claim.get_claim_by_id(gold_claims, cur_prediction.claim_id)
claim_predictions = ClaimPredictions.from_formatted_prediction(cur_prediction, gold_claim=cur_claim)
org_claim_predictions.append(claim_predictions)

TypeError: 'ClaimPredictions' object is not subscriptable

In [ ]:
#NONE
# Formatting all the predictions
org_claim_predictions = []
for cur_prediction in tqdm(original_claim_predictions_raw, desc="Formatting predictions"):
    cur_claim = Claim.get_claim_by_id(gold_claims, cur_prediction['id'])
    claim_predictions = ClaimPredictions.from_formatted_prediction(cur_prediction, gold_claim=cur_claim)
    org_claim_predictions.append(claim_predictions)

NameError: name 'original_claim_predictions_raw' is not defined

In [11]:
#NONE
# Attack and paraphrase claims
paraphrased_attack_results = []
for cur_claim in tqdm(org_claim_predictions, desc="Attacking and Paraphrasing claims"):
    result = paraphrase_attack.attack(0, cur_claim.gold, cur_claim.predictions)
    paraphrased_attack_results.append(result)

NameError: name 'org_claim_predictions' is not defined

100%|██████████| 2/2 [00:01<00:00,  1.61it/s]


Predictions for 84: Mammalian-resident macrophages are maintained autonomous of circulating monocyte.


100%|██████████| 2/2 [00:01<00:00,  1.68it/s]


Predictions for 84: Epithelial macrophages are maintained independent of circulating monocyte/matrix.


100%|██████████| 2/2 [00:01<00:00,  1.69it/s]


...

Predictions for 84: Microbial-resident macrophages are maintained independently of adult monocyte numbers.


100%|██████████| 2/2 [00:01<00:00,  1.68it/s]


Predictions for 84: Adult cells-resident macrophages are maintained independent of circulating monocyte and


100%|██████████| 2/2 [00:01<00:00,  1.35it/s]

Predictions for 84: Mammalian-resident macrophages are maintained for life independent of circulating monocyte count.


In [44]:
for cur_res in result:
    print(cur_res.original_prediction)

{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4]), 22406695: PredictedAbstract(abstract_id=22406695, label=<Label.SUPPORTS: 2>, rationale=[1]), 22973574: PredictedAbstract(abstract_id=22973574, label=<Label.SUPPORTS: 2>, rationale=[3])}
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4]), 22406695: PredictedAbstract(abstract_id=22406695, label=<Label.SUPPORTS: 2>, rationale=[1]), 22973574: PredictedAbstract(abstract_id=22973574, label=<Label.SUPPORTS: 2>, rationale=[3])}
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4]), 22406695: PredictedAbstract(abstract_id=22406695, label=<Label.SUPPORTS: 

{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.REFUTES: 0>, rationale=[]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4])}
{}
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4])}
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4]), 22973574: PredictedAbstract(abstract_id=22973574, label=<Label.SUPPORTS: 2>, rationale=[3])}
{}
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPORTS: 2>, rationale=[4])}
{}
{12827098: PredictedAbstract(abstract_id=12827098, label=<Label.SUPPORTS: 2>, rationale=[5]), 7521113: PredictedAbstract(abstract_id=7521113, label=<Label.SUPPO